<a href="https://colab.research.google.com/github/YaCpotato/AP/blob/master/TitanicExproration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Titanic Exproration
### link
https://www.kaggle.com/c/titanic

### reference

### Discription
This notebook is exploration of titanic dataset

In [1]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [2]:
!pip install kaggle

In [3]:
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c titanic

  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 24.9MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 38.7MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.82MB/s]


In [5]:
!unzip /content/train.csv
!unzip /content/test.csv

Archive:  /content/train.csv
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/train.csv or
        /content/train.csv.zip, and cannot find /content/train.csv.ZIP, period.
Archive:  /content/test.csv
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/test.csv or
        /content/test.csv.zip, and cannot find /content/test.csv.ZIP, period.


In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization

###################################use for kaggle notebooks######################################
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
#
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
!pip install pycaret

     |████████████████████████████████| 266kB 7.3MB/s 
     |████████████████████████████████| 14.2MB 291kB/s 
     |████████████████████████████████| 2.0MB 41.5MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 276kB 44.1MB/s 
     |████████████████████████████████| 215kB 47.3MB/s 
     |████████████████████████████████| 245kB 46.8MB/s 
     |████████████████████████████████| 1.7MB 44.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 102kB 9.6MB/s 
     |████████████████████████████████| 6.8MB 25.8MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 378kB 44.9MB/s 
     |████████████████████████████████| 163kB 45.5MB/s 
     |████████████████████████████████| 153kB 50.7MB/s 

In [58]:
# Read Train Data
train_data = pd.read_csv("/content/train.csv")
test_data = pd.read_csv("/content/test.csv")

test_data_y = pd.read_csv("/content/gender_submission.csv")

In [59]:
train_data = train_data.drop(['Name','Ticket','Cabin','Embarked'], axis = 1)
train_data = pd.get_dummies(train_data,columns=['Sex'], dtype = 'bool')
test_data = test_data.drop(['Name','Ticket','Cabin','Embarked'], axis = 1)
test_data = pd.get_dummies(test_data,columns=['Sex'], dtype = 'bool')

In [60]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
X_train = train_data.drop('Survived', axis = 1)
Y_train = train_data['Survived']
X_test = test_data
Y_test = test_data_y['Survived']

lgb_train = lgb.Dataset(X_train, Y_train)
lgb_eval = lgb.Dataset(X_test, Y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [64]:
print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
Y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(Y_test, Y_pred) ** 0.5)


Starting training...
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000117 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's l2: 0.219117	valid_0's l1: 0.45551
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's l2: 0.206762	valid_0's l1: 0.442335
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's l2: 0.196358	valid_0's l1: 0.430815
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's l2: 0.1861	valid_0's l1: 0.418857
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's l2: 0.177382	valid_0's l1: 0.408177
[LightGBM] [Warning] No further splits with positive gain, best gain